In [1]:
import numpy as np

a = np.asarray([[1,2,3],[4,5,6]]);
b = np.asarray([[10]]);

res = a*b;

print(res);

import sys;
print(sys.path)


[[10 20 30]
 [40 50 60]]
['', '/Users/norbertmauger/anaconda/envs/py35/lib/python35.zip', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/plat-darwin', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/lib-dynload', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/site-packages/Sphinx-1.3.5-py3.5.egg', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/site-packages/setuptools-20.3-py3.5.egg', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/site-packages', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/site-packages/aeosa', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/site-packages/IPython/extensions', '/Users/norbertmauger/.ipython']


In [11]:
import numpy as np;

import theano.tensor as T;
from theano import function;
from theano import pp;

x = T.scalar('x');
y = T.scalar('y');

z = x + y;
z2 = x * y;
print(pp(z));
f = function([x,y],z)
f2 = function([x,y],[z,z2])




(x + y)


In [35]:
print(f(2,3))
print(f2(2,3))
np.allclose(f(2.34,5.67), 8.01)

np.allclose([3,3.9,5], [3,4,5.1],atol=0.1)


5.0
[array(5.0), array(6.0)]


True

In [39]:
x = T.dmatrix('x');
y = T.dmatrix('y');

z = x + y;
f = function([x,y],z)

f([[2,3],[4,7]], [[30,10],[50,30]])

array([[ 32.,  13.],
       [ 54.,  37.]])

In [45]:
import theano
a = theano.tensor.vector()
b = theano.tensor.vector()
out = a**2 + b**2 + 2*a*b
f = theano.function([a,b],out)
f([3,1,34],[4,1,321])

array([  4.90000000e+01,   4.00000000e+00,   1.26025000e+05])

In [58]:
import theano
import theano.tensor as T
x = T.matrix('x')
s = 1 / (1 + T.exp(-x))

sigmoid = function([x], s)
sigmoid([[0,1,-1000],[2,3,1000]])

array([[ 0.5       ,  0.73105858,  0.        ],
       [ 0.88079708,  0.95257413,  1.        ]])

In [65]:
# possible de créer plusieurs variable de type matrice en même temps via dmatrices
# Possible ensuite de faire faire plusieurs calculs simultanément à une fonction
a, b = T.dmatrices('a','b')
diff = a - b
absdiff = abs(diff)
sqrdiff = diff**2

dif_func = function([a,b], [diff, absdiff, sqrdiff])

dif_func([[2,3]],[[2,5]])

[array([[ 0., -2.]]), array([[ 0.,  2.]]), array([[ 0.,  4.]])]

In [73]:
from theano import In
from theano import function

x,y = T.dscalars('x', 'y')
out = x + y

f = function([x,In(y, value = 1)], out)

print(f(3,2))
print(f(10))

5.0
11.0


In [78]:
x,y,w = T.dscalars('x', 'y', 'w')
out = (x+y)*w

f = function([x, In(y, value=10), In(w, value=1)], out)

print(f(2))
print(f(2,1))
print(f(2, w=5))
print(f(2,w=5,y=2))

12.0
3.0
60.0
20.0


In [93]:
from theano import shared

x = T.dscalar('x')
calls = shared(0)
out = x**3

f = function([x], out, updates=[(calls, calls+1)])

f(5)
print(calls.get_value())
f(5)
print(calls.get_value())
f(5)
print(calls.get_value())

1
2
3


In [94]:
import numpy
import theano
import theano.tensor as T
rng = numpy.random

N = 400                                   # training sample size
feats = 784                               # number of input variables

# generate a dataset: D = (input_values, target_class)
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

# Declare Theano symbolic variables
x = T.dmatrix("x")
y = T.dvector("y")

# initialize the weight vector w randomly
#
# this and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)
w = theano.shared(rng.randn(feats), name="w")

# initialize the bias term
b = theano.shared(0., name="b")

print("Initial model:")
print(w.get_value())
print(b.get_value())

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize
gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                          # (we shall return to this in a
                                          # following section of this tutorial)

# Compile
train = theano.function(
          inputs=[x,y],
          outputs=[prediction, xent],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])

print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Initial model:
[  2.30950367e-01  -1.05041231e+00  -3.32865926e-01   7.99277647e-01
  -2.04948422e-01  -6.80768708e-01  -1.45021411e+00   2.68211179e-01
  -1.25068502e+00   6.88195150e-01   1.22788058e-01  -7.54472416e-01
   2.03287069e-02  -1.09162707e+00   1.76384508e-01  -1.68590982e+00
   2.48922535e-01   4.04502174e-01   5.50463242e-01  -2.48636557e+00
  -1.38199545e+00  -1.16734315e+00   1.22903958e+00   1.16934515e+00
  -4.10201310e-01  -2.28005535e-02   1.54939160e+00   1.21207200e+00
  -1.24951634e+00  -9.62011454e-01   7.48441162e-04   1.27281886e-01
  -4.45804266e-01   8.65845582e-01   3.00082026e-01  -2.24925189e-01
   6.19771034e-02  -7.75948758e-01   1.74947069e-01  -1.07030314e+00
   4.10844174e-01  -1.46371421e+00   3.40555787e-01  -1.15783334e-01
   1.17863071e+00  -1.57673848e+00   1.06820218e+00  -7.32768276e-01
  -8.74494766e-01  -1.64827190e+00  -5.27363092e-02   3.09207792e-01
  -2.78774428e-01   1.02138787e-01  -3.27094604e-02   2.93294937e+00
  -1.54995492e+00  